In [1]:
import pandas as pd
from scripts import daily_balance as db
from pyxirr import xirr


In [2]:
# Funcion Original
def calculate_xirr(acct_balance_df: 'pd', contributions_df: 'pd', bal_column: str, cont_column: str) -> float:
    # Get last value from account balances
    last_balance_value = acct_balance_df.copy(deep=True)
    last_balance_value = last_balance_value.iloc[-1:]

    # Rename column to match contributions df
    last_balance_value.rename(
        columns={bal_column: cont_column},
        inplace=True)
    # Concatenate dataframes
    irr_df = pd.concat([contributions_df, last_balance_value])
    # Separate df into two lists
    pd_List1 = irr_df.index.tolist()
    pd_List2 = list(irr_df[cont_column])
    # Pass lists into xirr function
    xirr_result = xirr(pd_List1, pd_List2)
    return xirr_result

In [50]:
# Funcion ejecutada directamente
acct_balance_df = db.create_df('outputs/irr_monthly_account_balance_PCL_AllAccounts.csv')
contributions_df = db.create_df('outputs/irr_contributions_PCL_AllAccounts.csv')
bal_column = 'Tot_Acct_Portafolio_MXN'
cont_column = 'Contribuciones_Totales_MXN'  # Antes 'Tot_Contribuciones_MXN'

# Get last value from account balances
last_balance_value = acct_balance_df.iloc[-1:]

# Rename column to match contributions df
last_balance_value = last_balance_value.rename(columns={bal_column: cont_column})

# # Concatenate dataframes
irr_df = pd.concat([contributions_df, last_balance_value])

# # Separate df into two lists
date_list = irr_df.index.tolist()
values_list = list(irr_df[cont_column])

# # Pass lists into xirr function
xirr_result = xirr(pd_List1, pd_List2)

irr_df

Date,Contribuciones_Totales_MXN
2018-04-24,-100000
2018-07-09,-9000
2018-09-09,-14000
2018-09-22,-4000
2018-10-31,103701
2020-03-03,30361
2018-01-30,-100000
2018-05-17,-50000
2018-06-04,-30000
2018-06-11,-40000


In [47]:
# Hardcoded Inputs
acct_balance_df = db.create_df('outputs/irr_monthly_account_balance_PCL_AllAccounts.csv')
contributions_df = db.create_df('outputs/irr_contributions_PCL_AllAccounts.csv')
bal_column = 'Tot_Acct_Portafolio_MXN'
cont_column = 'Contribuciones_Totales_MXN'  # Antes 'Tot_Contribuciones_MXN'

In [49]:
# Function1
def last_value(df):
    df = df.iloc[-1:]
    return df

a = last_value(acct_balance_df)
a

Date,Tot_Acct_Portafolio_MXN
2022-06-30,1138511


In [52]:
# Function2
def rename_column(df, balance_column, contributions_column):
    df = df.rename(columns={balance_column: contributions_column})
    return df

b = rename_column(a, bal_column, cont_column)
b

Date,Contribuciones_Totales_MXN
2022-06-30,1138511


In [53]:
# Concatenate dataframes
irr_df = pd.concat([contributions_df, last_balance_value])
irr_df

Date,Contribuciones_Totales_MXN
2018-04-24,-100000
2018-07-09,-9000
2018-09-09,-14000
2018-09-22,-4000
2018-10-31,103701
2020-03-03,30361
2018-01-30,-100000
2018-05-17,-50000
2018-06-04,-30000
2018-06-11,-40000


In [ ]:
# Function3 Separate into two lists
def cf_tolist(df, contributions_column):
    # Separate df into two lists
    date_list = df.index.tolist()
    values_list = list(df[contributions_column])
    return date_list, values_list

